In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np

In [ ]:
df1=pd.read_csv("../data/NSSCrisisv.1.csv")
df1.head()

In [ ]:
header_row = df1.iloc[1]
df1 = pd.DataFrame(df1.values[2:], columns=header_row)
df1.head()

In [ ]:
df2=pd.read_csv("../data/NSSCrisisv.2.csv")
df2.head(5)

In [ ]:
header_row = df2.iloc[1]
df2 = pd.DataFrame(df2.values[2:], columns=header_row)
df2.head()

In [ ]:
crisis_call=pd.ExcelFile("../data/NSS_Crisis_call.xlsx")
volume2020 = pd.read_excel(crisis_call, "2020")
volume2021 = pd.read_excel(crisis_call, "2021")
volume2022 = pd.read_excel(crisis_call, "2022")
volume2020.head()

In [ ]:
df1.info()

## Who works at the call center?

In [ ]:
workers=df1.PhoneWorkerName.unique()
workers

## How many calls did each worker take?

In [ ]:
work_call_vol=df1.PhoneWorkerName.value_counts()
work_call_vol

## Lets pull fix the dates and time

In [ ]:
df1.CallDateAndTimeStart	=df1.CallDateAndTimeStart	.str.split(" ")
df1.head(3)

## Close enough for horseshoes and hand grenades, so let's roll with it.
#### In the mean time, lets look at the average time each worker spent on a call. Does that yeild better results?

In [ ]:
df1.CallLength=df1.CallLength.astype('float')
df1.groupby('PhoneWorkerName').CallLength.mean()

In [ ]:
samecolumnsdf=pd.DataFrame(df1.columns.intersection(df2.columns))
# something Maggie shared. Maybe it's the merge we need right now
samecolumnsdf
#Nope, don't like that.

In [ ]:
df1.WasPrankCall.unique()
#what good is a column that is entirely null?

In [ ]:
pct_null = df1.isnull().sum() / len(df1)
missing_features = pct_null[pct_null > 0.98].index
df1.drop(missing_features, axis=1, inplace=True)
#Rob Schulteis found a way to drop columns that were 98%+ null

In [ ]:
df1.shape[1]
#87 columns is much more managable than 300 something

In [ ]:
df1.rename(columns={'CRISIS Demographics - Ethnicity': 'Ethnicity'}, inplace=True)
df2.rename(columns={'CRISIS Demographics - Race/Ethnicity': 'Ethnicity'}, inplace=True)
df1.Ethnicity.unique()

In [ ]:
df1.Ethnicity.value_counts()

In [ ]:
df1['CallDateAndTimeStart'].head()

#### I do not like these brackets. It makes this impossible to split. Let's ditch them.

In [ ]:
df1['CallDateAndTimeStart'] = df1['CallDateAndTimeStart'].str.get(0)
df1.head()

In [ ]:
roe=df1['CRISIS Call Information - Call Type'].value_counts()
roe=pd.DataFrame(roe)
roe

In [ ]:
roe.reset_index(inplace=True)
roe=roe.rename(columns={'index':'Call_Type', 'CRISIS Call Information - Call Type':'num'})
roe

In [ ]:
roe.info()

In [ ]:
df1['CRISIS Demographics - Education Level'].value_counts()

In [ ]:
df1['CRISIS Issues - Mental Health'].value_counts()

### Okay, enough of the value counts. Time to move on to somehting else.

In [ ]:
df1.Feedback.unique()

In [ ]:
volume2020

### 107  Spanish calls in 2020

In [ ]:
volume2021

### 248 crisis in Spanish in 2021

In [ ]:
volume2022

In [ ]:
### Spanish 65 Jan-Apr
65*4

###### 2020=107
2021=248
2022=260

In [ ]:
v20=pd.DataFrame(volume2020)
v21=pd.DataFrame(volume2021)

In [ ]:
#Pandemic_years=pd.concat(v20, v21, keys="Report Info")
#Pandemic_years

#### Analyze trends in call content by looking for which issues are most frequent and what is the average count of issues per call.

In [ ]:
labels = list()
for col in df2.columns:
    labels.append(col.replace('Subjective', 'Issues'))
df2.columns = labels
#for loop to change titles to match version 1 submitted by Rob S.

In [ ]:
df2.isnull().sum()

In [ ]:
df1.isnull().sum()

In [ ]:
df1_ci = df1[['CRISIS Issues - Abusive Behavior','CRISIS Issues - Emotional State','CRISIS Issues - Financial/Basic Needs','CRISIS Issues - Health/Physical','CRISIS Issues - Information or Services Needed','CRISIS Issues - Mental Health','CRISIS Issues - No Issue Call','CRISIS Issues - Other Description','CRISIS Issues - Relationships','CRISIS Issues - Substances','CRISIS Issues - Suicide']]
df1_ci.head()
#Jasmine Drumright narrows down to just Crisis issues.

In [ ]:
filter_col = [col for col in df2 if col.startswith('CRISIS Issues') or col.startswith('Call')]
df2_ci=df2[filter_col]
df2_ci.head()
#Rob's version of what Jasmine did for the second dataframe 

In [ ]:
Cri=pd.concat([df1_ci, df2_ci])
Cri

In [ ]:
Cri['CRISIS Issues - Emotional State'].value_counts()

In [ ]:
Cri['CRISIS Issues - Health/Physical'].value_counts()

In [ ]:
#Cri['CRISIS Issues - Financial/Basic Needs'].value_counts()
percent100 = Cri['CRISIS Issues - Financial/Basic Needs'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
percent100

In [ ]:
#I have continued to add to this list as problems developed later down the line. 
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace('(temporary)','')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace('(permanent)','')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace(r' ; \w+','')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace('overdraft or other issues','')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace(r'\(\)','')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace('/',' or ')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace('Housing ','Housing')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace('Housing; Housing','Housing')

per_fin=Cri['CRISIS Issues - Financial/Basic Needs'].value_counts()#normalize=True).mul(100).round(1).astype(str) + '%'
per_fin

###### Tried to insert the melt madness here. It didn't work. Stop jumping around and leave sleeping code lie. 

In [ ]:
per_fin=Cri['CRISIS Issues - Financial/Basic Needs'].value_counts()
per_fin

### Below is a loop of counts per item across columns by Rob Schulteis

In [ ]:
mental_health_df = Cri['CRISIS Issues - Mental Health'].notnull()

In [ ]:
mental_health_df = Cri[mental_health_df]

In [ ]:
mental_health_df.head(100)

In [ ]:
count = []

for x in mental_health_df['CRISIS Issues - Mental Health']:
    counter0 = x.count(";")+1
    count.append(counter0)

In [ ]:
mental_health_df['MH_Issues_Count'] = count

In [ ]:
mental_health_df.head()

### Finished with that Loop, now back to parsing out the data

#### I suppose the next thing I ought to do is create a dataframe out of my drill down so that I can use it in a chart. Let's try

In [ ]:
per_fin=pd.DataFrame(per_fin)
per_fin.reset_index(inplace=True)
per_fin = per_fin.rename(columns = {'index':"Need", 'CRISIS Issues - Financial/Basic Needs':"Num_of_calls"})
print(per_fin)

In [ ]:
sns.set_theme(style="darkgrid")
sns.set(font_scale=1)
sns.barplot(x="Need", y="Num_of_calls", data=per_fin.head(5), palette="viridis", edgecolor='black')
plt.title('Top 5 needs by call number', fontsize=20)
plt.xticks(rotation=50, fontsize=16)
plt.xlabel('', fontsize=0)
plt.ylabel('number of calls by volume', fontsize=16)
plt.yticks(fontsize=16)
plt.gcf().set_size_inches(15,7)

#### Okay, I like where I'm at with that. We have a chart and it looks good. The next Crisis we should drill down into should be...
###### Abuse

In [ ]:
Cri['CRISIS Issues - Abusive Behavior'].value_counts()

In [ ]:
#We have to set the stage again or else things won't work later on.
Cri['CRISIS Issues - Abusive Behavior'] = Cri['CRISIS Issues - Abusive Behavior'].str.replace(';',' conjoined with')
Cri['CRISIS Issues - Abusive Behavior'] = Cri['CRISIS Issues - Abusive Behavior'].str.replace('/',' or ')
abuse=Cri['CRISIS Issues - Abusive Behavior'].value_counts()

In [ ]:
abuse=pd.DataFrame(abuse)
abuse.reset_index(inplace=True)
abuse = abuse.rename(columns = {'index':"type_of_abuse", 'CRISIS Issues - Abusive Behavior':"count"})
print(abuse)

In [ ]:
sns.set_theme(style="whitegrid")
sns.set(font_scale=1)
sns.barplot(y="type_of_abuse", x="count", data=abuse.head(10), palette=['pink','magenta'], edgecolor='black', orient='h')
plt.title('Of those who suffered abuse', fontsize=20)
plt.xticks(rotation=0, fontsize=12)
plt.xlabel('type', fontsize=0)
plt.ylabel('number of abused callers', fontsize=16)
plt.yticks(fontsize=16)
plt.gcf().set_size_inches(15,7)

### Going back to per_fin I would like to find the average call time per issue
##### Working with Nathan to get that done.

In [ ]:
per_fin.Need

In [ ]:
# count=[]
# Housing=0
# Employment=0
# Bank=0
# Medication= 0
# Car=0
# Rent=0
# Food=0
# Utilities=0
# Home_Repair=0
# Food=0
# Insurance=0
# Moving=0
# Legal=0
# Transportation=0
# Clothing=0
# Credit=0
# Crime=0
# Taxes=0
# Child=0
# Keeping=0
# Rehab=0

# for x in mental_health_df['CRISIS Issues - Financial/Basic Needs']:
#     counter0 = x.count(";") + 1
#     Needs_sum= Needs_sum + counter0
#     hous_c = x.count('Housing')
#     hous_sum = hous_sum + hous_c
#     employment_c = x.count("Employment")
#     emp_sum = emp_sum + employment_c
#     bank_c = x.count('Bank Account')
#     bank_sum = bank_sum + bank_c
#     med_c = x.count("Medication")
#     med_sum = med_sum + med_c
#     car_c = x.count('Car Repair')
#     car_sum = car_sum + car_c
#     rent_c = x.count("Rent")
#     rent_s = rent_s +rent_c
#     food_c = x.count("Food")
#     food_s = food_s + food_c
#     util_c = x.count("Utilities")
#     util_s = util_s + util_c
#     home_c = x.count("Home Repair")
#     home_s = home_s + home_c
#     insur_c = x.count("Insurance")
#     insur_s = insur_s + insur_c
#     move_c = x.count("Moving Assistance")
#     move_s = move_s + move_c
#     legal_c = x.count("Legal Services")
#     legal_s = legal_s +legal_c
#     trans_c = x.count("Transportation")
#     trans_s = trans_s + trans_c
#     cloth_c = x.count("Clothing")
#     cloth_s = cloth_s + cloth_c
#     credit_c = x.count('Credit Card Debt')
#     credit_s = credit_s + credit_c
#     fisc_c = x.count('Financial Crime')
#     fisc_s = fisc_s + fisc_c
#     tax = x.count('Taxes')
#     is_theft = is_theft + tax
#     child_c = x.count('Child Support')
#     child_s = child_s + child_c
#     keep = x.count('House Keeping')
#     keeping = keeping + keep
#     drug = x.count('Drug Rehab')
#     rehab = rehab + drug
    
# Need_Sumdf = {'Need': ['Housing','Employment','Bank Account','Medication'
# 'Car Repair',
# 'Rent\Mortgage',
# 'Food',
# 'Utilities',
# 'Home Repair',
# 'Food',
# 'Insurance',
# 'Moving Assistance',
# 'Legal Services',
# 'Transportation',
# 'Clothing',
# 'Credit Card Debt',
# 'Financial Crime',
# 'Taxes',
# 'Child Support',
# 'House Keeping',
# 'Drug Rehab'],
# "Sum": ['hous_sum',
#     'emp_sum',
#     'bank_sum', 
#     'med_sum',
#     'car_sum',
#     'rent_s',
#     'food_s',
#     'util_s',
#     'home_s',
#     'insur_s',
#     'move_s',
#     'legal_s',
#     'trans_s',
#     'cloth_s',
#     'credit_s',
#     'fisc_s',
#     'is_theft',
#     'child_s',
#     'keeping',
#     'rehab']
    
#      Need_Sumdf = pd.DataFrame(Need_Sumdf)
#      Need_Sumdf = Need_Sumdf.sort_values(by='Sum', ascending=False).reset_index(drop=True)
# #Mental_Health_Sum_df_10 = Mental_Health_Sum_df.nlargest(10, 'Sum').sort_values(by='Sum', ascending=False)
#      Need_Sumdf

## Spent way too much time on that and it's off

In [ ]:
new_df=per_fin['Need'].str.split('; ', expand=True)
new_df

In [ ]:
new_df=pd.DataFrame(new_df)
new_df.reset_index(inplace=True)

In [ ]:
new_df

In [ ]:
melted=new_df.melt().value.value_counts() 
melted

In [ ]:
melted=pd.DataFrame(melted)
melted.reset_index(inplace=True)
melted.dropna()

In [ ]:
melted['index'] = melted['index'].str.replace(' ','_')

In [ ]:
melted

#### Now I can actually see how they are labeled differently and adjust or merge columns

In [ ]:
#This worked. Do I have to move it to before the melt? 
#Answer: No, don't do that
melted['index'] = melted['index'].str.replace(r'Bank_Acco\w+','Bank_Account')
melted['index'] = melted['index'].str.replace('Home_repair_or_maintenance','Home_Repair')
melted['index'] = melted['index'].str.replace('Payment_towards_Lawyer_or_Legal_Services', 'Legal_Fees')
melted['index'] = melted['index'].str.replace(r'Credit_Card_Debt\s\w*?', 'Credit_Card_Debt')
melted['index'] = melted['index'].str.replace('PBank_Account__Theft_or_Fraud_or_Financial_Crime', 'Financial_Crime')
melted['index'] = melted['index'].str.replace('Identity_Theft_or_Fraud_or_Financial_Crime', 'Financial_Crime')

In [ ]:
smelted=melted.groupby(['index'],as_index=False).sum()
smelted = smelted.sort_values(by = ['value'], ascending=[False])
print(smelted)

### Progress! Got it to work, now I need it ordered